In [ ]:
import numpy as np
import gmm_library as gmm
import moments_function_library as mfl

from scipy import stats
from scipy.stats import norm
from scipy.integrate import simpson
from scipy.optimize import minimize
from numpy.polynomial import Polynomial
from matplotlib import pyplot as plt

In [ ]:
# -----------------------------------------------------------------------------------------------------------------
# Functions
# -----------------------------------------------------------------------------------------------------------------

def create_return_wave(scatter, func, noise=None, add_noise=False, snr=100, real=True, plot=False):
    composite = func.create_composite(scatter)
    if real:
        composite = np.real(composite)
    composite = func.normalize(composite)
    if add_noise:
        if noise is None:
            if real:
                noise = np.random.normal(0, 1/snr, len(func.t))
            else:
                noise = np.random.normal(0, 1/snr, len(func.t)) + 1j*np.random.normal(0, 1/snr, len(func.t))
        composite += noise
    if plot:
        print(np.sqrt(np.mean(np.abs(noise)**2)))
        plt.plot(func.t, np.real(composite))
        plt.show()
        print(np.sqrt(np.mean(np.abs(composite)**2)))
    return composite

def poly_fit(t, return_sig, FitWindow=1, SampleRate=1000, power=4, real=True, segments=3):
    if real:
        return_sig  = np.real(return_sig)
    SignalLen   = len(return_sig)
    WindowLen   = SampleRate*FitWindow
    poly_coefs  = []
    for _ in range(segments):
        t1          = int((SignalLen-WindowLen+(WindowLen*_))/2)
        t2          = int((SignalLen+WindowLen+(WindowLen*_))/2)
        fitted_poly = Polynomial.fit(t[t1:t2], return_sig[t1:t2], power, domain=[t[0], t[-1]])
        for i in fitted_poly.convert().coef:    # Coefficients in ascending order
            poly_coefs.append(i)
    return poly_coefs

def add_poly_coefs(row, base, power, FitWindow=1, add_noise=False, real=True, snr=100,
                    segments=3, freqs=6, plot=False, SampleRate=1000):
    # create scatter distribution as defined by the locations and amplitudes of the delta functions
    scatter         = np.zeros(2*int(SampleRate))
    scatter[int((row['loc_1']+1) * int(SampleRate))] = row['amp_1']
    scatter[int((row['loc_2']+1) * int(SampleRate))] = row['amp_2']
    scatter[int((row['loc_3']+1) * int(SampleRate))] = row['amp_3']
    scatter        /= np.sum(scatter)
    # create return functions by convolving outbound waves with scatter distribution
    base.return_sig = create_return_wave(scatter, base, plot=plot,
                                         add_noise=add_noise, snr=snr, real=real)
    poly_coefs      = poly_fit(base.t, base.return_sig, SampleRate=SampleRate, real=real,
                                        power=power, FitWindow=FitWindow, segments=segments)
    f               = np.fft.rfft(np.real(base.return_sig)) if real else np.fft.fft(base.return_sig)
    indexed_values  = list(enumerate(f))
    top_six         = sorted(sorted(indexed_values, key=lambda x: abs(x[1]), reverse=True)[:freqs],
                            key=lambda x: x[0])
    f_complex       = [val for idx, val in top_six]
    f_amps          = np.abs(f_complex)
    f_phases        = np.angle(f_complex)
    f_phases[::2]  -= np.pi
    f_unwrapped     = np.unwrap(f_phases)
    f_slope, _      = np.polyfit([i for i in range(len(f_unwrapped))], f_unwrapped, 1)
    final_array     = np.array([row['loc_1'], row['loc_2'], row['loc_3'], row['amp_1'], row['amp_2'], row['amp_3'],
                                row['mean'], row['variance'], row['std'], row['skew'], row['kurtosis'], row['inv_kurt'],
                                row['hyperskewness'], row['hyperkurtosis']])
    final_array     = np.append(final_array, poly_coefs)
    final_array     = np.append(final_array, f_amps)
    final_array     = np.append(final_array, f_slope)
    return final_array

def add_poly_coefs_norm(row, base, power, FitWindow=1, add_noise=False, real=True, snr=100,
                         segments=3, freqs=6, plot=False, threshold=1e6, SampleRate=1000):
    target_moments      = [row['mean'], row['variance'], row['std'], row['skew'], row['kurtosis']]
    results, moms, _    = gmm.match_moments(target_moments)
    x                   = np.linspace(-2, 2, 4*int(SampleRate))
    scatter             = gmm.gmm_pdf(x, results)
    scatter            /= np.sum(scatter)
    moments             = mfl.Moments(x, scatter).moments
    # make sure the gmm's moments match the target moments, otherwise the record is unusable
    # if np.array_equal(np.round(moments[:5], 2), np.round(target_moments[:5], 2)):
    final_array     = np.append(results, moments)
    base.return_sig = create_return_wave(scatter, base, plot=plot, add_noise=add_noise, snr=snr, real=real)
    poly_coefs      = poly_fit(base.t, base.return_sig, SampleRate=SampleRate, real=real,
                                    power=power, FitWindow=FitWindow, segments=segments)
    f               = np.fft.rfft(np.real(base.return_sig)) if real else np.fft.fft(base.return_sig)
    indexed_values  = list(enumerate(f))
    top_six         = sorted(sorted(indexed_values, key=lambda x: abs(x[1]), reverse=True)[:freqs],
                            key=lambda x: x[0])
    f_complex       = [val for idx, val in top_six]
    f_amps          = np.abs(f_complex)
    f_phases        = np.angle(f_complex)
    f_phases[::2]  -= np.pi
    f_unwrapped     = np.unwrap(f_phases)
    f_slope, _      = np.polyfit([i for i in range(len(f_unwrapped))], f_unwrapped, 1)
    # else:
    #     final_array     = np.append(results, [np.nan for _ in range(len(moments))])
    #     poly_coefs      = [np.nan for _ in range((power+1)*segments)]
    #     f_amps          = [np.nan for _ in range(freqs)]
    #     f_slope         = np.nan
    final_array         = np.append(final_array, poly_coefs)
    final_array         = np.append(final_array, f_amps)
    final_array         = np.append(final_array, f_slope)
    return final_array